In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from sbi_ice.simulators import Layer_Tracing_Sim as lts
from sbi.utils import process_prior
from sbi_ice.utils import custom_priors
from tueplots import figsizes
from sbi_ice.utils import plotting_utils,misc
data_dir,output_dir = misc.get_data_output_dirs()

color_opts = plotting_utils.setup_plots()

# LMI Body Figure

Here we demonstrate what the LMI boundary is in our simulations (appendix figure).

In [ ]:
#Load synthetic shelf data

data_file = Path(data_dir,"Synthetic_long","setup_files","flowtube_tmb_final.csv")

df = pd.read_csv(data_file)
xmb = df["x_coord"].to_numpy() #x - coordinates of domain
tmb = df["tmb"].to_numpy() #total mass balance
Lx = xmb.max() - xmb.min()

nx_iso             = 500 # Number of points in the x-direction
ny_iso             = 1 # Number of points in the y-direction
dt                 = 0.5 # [yr] timestep for advection scheme


geom = lts.Geom(nx_iso=nx_iso,ny_iso=ny_iso) #This will be the main object we interact with





## Define SMB (and BMB) profiles

In [ ]:
torch.manual_seed(400) #Set the seed for reproducibility

#We use a perturbed GP prior. First, we have a regular GP prior with mean 0, variance 1, and a specified lengthscale (with a Matern kernel).
#We then perturb the GP samples by multiplying them by a variance factor and adding a constant mean, which are independently sampled from specified distributions.


GP_mean_mu= 0.5 #If GP mean sampled from a Gaussian, this is the mean of that Gaussian
GP_mean_sd= 0.25 #If GP mean sampled from a Gaussian, this is the standard deviation of that Gaussian
# GP_mean_min = 0.0 #If GP mean sampled from a uniform, this is the minimum of that uniform
# GP_mean_max = 1.0 #If GP mean sampled from a uniform, this is the maximum of that uniform
GP_var_min= 0.1 #Minimum variance of GP
GP_var_max= 0.3 #Maximum variance of GP
length_scale= 2.5E+3 #Length scale of GP
nu= 2.5 #Smoothness of GP

ker =  Matern(length_scale=length_scale,nu=nu) #Matern kernel as Gaussian is too smooth
gpr = GaussianProcessRegressor(kernel=ker)
mvn_mean,mvn_cov = gpr.predict(xmb.reshape(-1,1),return_cov=True) #We take the usual GP marginalized covariance on our grid, and make sure it is positive definite by adding a small diagonal matrix
eps = 1e-6
a = np.zeros(shape = mvn_cov.shape)
np.fill_diagonal(a,eps)
mvn_cov += a
mvn_mean = torch.from_numpy(mvn_mean)
mvn_cov = torch.from_numpy(mvn_cov)
custom_prior = custom_priors.AppendedMVN(torch.tensor([GP_mean_mu]),torch.tensor([GP_mean_sd]),torch.tensor([GP_var_min]),
                                            torch.tensor([GP_var_max]),mvn_mean,mvn_cov)

# This is a function from the sbi utils, making it easier to sample from the prior and enforces strict bounds on the prior.
prior, *_ = process_prior(custom_prior,
                            custom_prior_wrapper_kwargs=dict(lower_bound=torch.cat((torch.tensor([GP_mean_mu-3*GP_mean_sd,GP_var_min]),-5.0*torch.ones(mvn_mean.size()))), 
                                                            upper_bound=torch.cat((torch.tensor([GP_mean_mu+3*GP_mean_sd,GP_var_max]),5.0*torch.ones(mvn_mean.size())))))


# prior, *_ = process_prior(custom_prior,
#                             custom_prior_wrapper_kwargs=dict(lower_bound=torch.cat((torch.tensor([GP_mean_min,GP_var_min]),-2.0*torch.ones(mvn_mean.size()))), 
#                                                             upper_bound=torch.cat((torch.tensor([GP_mean_max,GP_var_max]),2.0*torch.ones(mvn_mean.size())))))

#We sample a lot of smbs from this prior for plotting purposes, but only take one to use in the simulation.
samples = custom_prior.sample((1,)).numpy()
bmb_list = []
smb_cnst_mean_array = samples[:,0]
smb_sd_array= samples[:,1]
smb_unperturbed_array = samples[:,2:]
smb_samples = np.expand_dims(smb_cnst_mean_array,1) + np.expand_dims(smb_sd_array,1)*smb_unperturbed_array

smb_cnst_mean = smb_cnst_mean_array[0].copy()
smb_sd = smb_sd_array[0].copy()
smb_unperturbed = smb_unperturbed_array[0].copy()
smb = smb_samples[0].copy()

#Plot the smb samples
fig,ax = plt.subplots(1,1,figsize = (4,2))
ax.plot(xmb,smb_samples[:20].T,color = plotting_utils.color_opts["colors"]["prior"],alpha = 0.3)
#Finally, set the smb and bmb fields of the geom object.
smb_regrid,bmb_regrid = lts.init_geom_from_fname(geom,data_file,smb=smb)

# Simulate layers

In [ ]:
n_surface_phase1 = 25
n_base_phase1 = 100
time_phase1 = 500
n_surface_phase2 = 2
n_base_phase2 = 20
time_phase2 = 500
n_surface_phase3 = 1
n_base_phase3 = 1
time_phase3 = 50

sched1 = lts.Scheduele(time_phase1,dt,n_surface_phase1,n_base_phase1)
sched2 = lts.Scheduele(time_phase2,dt,n_surface_phase2,n_base_phase2)
sched3 = lts.Scheduele(time_phase3,dt,n_surface_phase3,n_base_phase3)

In [ ]:
#Now simulate

geom.initialize_layers(sched1,10)
lts.sim(geom,smb_regrid,bmb_regrid,sched1)
#Can save intermediate results here if we want
geom.initialize_layers(sched2,0)
lts.sim(geom,smb_regrid,bmb_regrid,sched2)
geom.initialize_layers(sched3,0)
lts.sim(geom,smb_regrid,bmb_regrid,sched3)

## Plot

In [ ]:
plt.rcParams.update(figsizes.icml2022_full(height_to_width_ratio=0.75))


fig,ax = plt.subplots(1,1)
ages = [50,200,300]
offsets = [-12,-12,-12]
age_indices = []
midway = int(geom.age_iso.size/2)
for age in ages:
    age_indices.append(midway + np.argmin(np.abs(geom.age_iso[midway:]-age)))
plotting_utils.plot_layers(geom.x,geom.bs.flatten(),geom.ss.flatten(),geom.dsum_iso[:,0,age_indices],geom.age_iso[age_indices],trackers=geom.extract_active_trackers(),ax=ax,color=plotting_utils.color_opts["colors"]["boundary_condition"])
for i,age in enumerate(age_indices):
    ax.annotate("age = {:.0f}a".format(geom.age_iso[age]),xy=(0,geom.bs.flatten()[0]+geom.dsum_iso[0,0,age]),xycoords="data",textcoords="offset points",xytext=(0,offsets[i]))

ax.set_xlabel("Distance [km]")
ax.set_ylabel("Elevation [m.a.s.l.]")
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.spines['bottom'].set_bounds(0-0.001,geom.x[-1]/1e3+0.001)

fig_name = Path(output_dir,"paper_figures","LMI_Body","LMI_Body.pdf")
fig_name.parent.mkdir(parents=True,exist_ok=True)
fig.savefig(fig_name)

